# Getting Started Guide

## Using Coach from the Command Line

When running Coach from the command line, we use a Preset module to define the experiment parameters.
As its name implies, a preset is a predefined set of parameters to run some agent on some environment.
Coach has many predefined presets that follow the algorithms definitions in the published papers, and allows training some of the existing algorithms with essentially no coding at all. This presets can easily be run from the command line. For example:

`coach -p CartPole_DQN`

You can find all the predefined presets under the `presets` directory, or by listing them using the following command:

`coach -l`

Coach can also be used with an externally defined preset by passing the absolute path to the module and the name of the graph manager object which is defined in the preset: 

`coach -p /home/my_user/my_agent_dir/my_preset.py:graph_manager`

Some presets are generic for multiple environment levels, and therefore require defining the specific level through the command line:

`coach -p Atari_DQN -lvl breakout`

There are plenty of other command line arguments you can use in order to customize the experiment. A full documentation of the available arguments can be found using the following command:

`coach -h`

## Using Coach as a Library

Alternatively, Coach can be used a library directly from python. As described above, Coach uses the presets mechanism to define the experiments. A preset is essentially a python module which instantiates a `GraphManager` object. The graph manager is a container that holds the agents and the environments, and has some additional parameters for running the experiment, such as visualization parameters. The graph manager acts as the scheduler which orchestrates the experiment.

Let's start with some examples.

Creating a very simple graph containing a single Clipped PPO agent running with the CartPole-v0 Gym environment:

In [ ]:
from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule

graph_manager = BasicRLGraphManager(
    agent_params=ClippedPPOAgentParameters(),
    env_params=GymVectorEnvironment(level='CartPole-v0'),
    schedule_params=SimpleSchedule()
)

Running the graph according to the given schedule:

In [ ]:
graph_manager.improve()

### Running each phase manually

The graph manager simplifies the scheduling process by encapsulating the calls to each of the training phases. Sometimes, it can be beneficial to have a more fine grained control over the scheduling process. This can be easily done by calling the individual phase functions directly:

In [ ]:
from rl_coach.core_types import EnvironmentSteps

graph_manager = BasicRLGraphManager(
    agent_params=ClippedPPOAgentParameters(),
    env_params=GymVectorEnvironment(level='CartPole-v0'),
    schedule_params=SimpleSchedule()
)

graph_manager.heatup(EnvironmentSteps(100))
graph_manager.train_and_act(EnvironmentSteps(100))

### Changing the default parameters

Agents in Coach are defined along with some default parameters that follow the published paper definition. This may be sufficient when running the exact same experiments as in the paper, but otherwise, there would probably need to be some changes made to the algorithm parameters. Again, this is easily modifiable, and all the internal parameters are accessible from within the preset:

In [ ]:
from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule
from rl_coach.graph_managers.graph_manager import ScheduleParameters
from rl_coach.core_types import TrainingSteps, EnvironmentEpisodes, EnvironmentSteps

# schedule
schedule_params = ScheduleParameters()
schedule_params.improve_steps = TrainingSteps(10000000)
schedule_params.steps_between_evaluation_periods = EnvironmentSteps(2048)
schedule_params.evaluation_steps = EnvironmentEpisodes(5)
schedule_params.heatup_steps = EnvironmentSteps(0)

# agent parameters
agent_params = ClippedPPOAgentParameters()
agent_params.algorithm.discount = 1.0

graph_manager = BasicRLGraphManager(
    agent_params=agent_params,
    env_params=GymVectorEnvironment(level='CartPole-v0'),
    schedule_params=schedule_params
)

graph_manager.improve()


### Using a custom gym environment

We can use a custom gym environment without registering it. 
We just need the path to the environment module.
We can also pass custom parameters for the environment `__init__` function as `additional_simulator_parameters`.

In [ ]:
from rl_coach.agents.clipped_ppo_agent import ClippedPPOAgentParameters
from rl_coach.environments.gym_environment import GymVectorEnvironment
from rl_coach.graph_managers.basic_rl_graph_manager import BasicRLGraphManager
from rl_coach.graph_managers.graph_manager import SimpleSchedule
from rl_coach.architectures.embedder_parameters import InputEmbedderParameters

# define the environment parameters
bit_length = 10
env_params = GymVectorEnvironment(level='rl_coach.environments.toy_problems.bit_flip:BitFlip')
env_params.additional_simulator_parameters = {'bit_length': bit_length, 'mean_zero': True}

# Clipped PPO
agent_params = ClippedPPOAgentParameters()
agent_params.network_wrappers['main'].input_embedders_parameters = {
    'state': InputEmbedderParameters(scheme=[]),
    'desired_goal': InputEmbedderParameters(scheme=[])
}

graph_manager = BasicRLGraphManager(
    agent_params=agent_params,
    env_params=env_params,
    schedule_params=SimpleSchedule()
)

In [ ]:
graph_manager.improve()

The path to the environment can also be set as an absolute path, as follows: `<absolute python module path>:<environment class>`. For example:

In [ ]:
env_params = GymVectorEnvironment(level='/home/user/my_environment_dir/my_environment_module.py:MyEnvironmentClass')